In [1]:
# 📦 Import des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 📂 Chargement des fichiers CSV (chemins nettoyés)
df_facture = pd.read_csv("C:/Users/User/Downloads/factures_fact.csv", sep=',')
df_admission = pd.read_csv("C:/Users/User/Downloads/admission_fact.csv", sep=',')
df_patient = pd.read_csv("C:/Users/User/Downloads/dim_patient.csv", sep=',')
df_cout = pd.read_csv("C:/Users/User/Downloads/dim_cout.csv", sep=',')

# 👀 Affichage des premières lignes
print("🧾 FACTURE")
display(df_facture.head())

print("\n🏥 ADMISSION")
display(df_admission.head())

print("\n🧑‍⚕️ PATIENT")
display(df_patient.head())

print("\n💸 TYPE DE COUT")
display(df_cout.head())

🧾 FACTURE


,patient_id,length_of_stay,facility_cost,procedure_cost,medication_cost,lab_test_cost,total_cost,subject_id,hadm_id
0,10006,8,8000,10500,21600,196300,236400,10006,142345
1,10011,13,13000,3000,0,70000,86000,10011,105331
2,10013,2,2000,1500,3600,14800,21900,10013,165520
3,10017,8,8000,3000,42000,47700,100700,10017,199207
4,10019,0,0,6000,0,28700,34700,10019,177759



🏥 ADMISSION


,subject_id,hadm_id,hospital_expire_flag,admission_count,procedure_count,los,Time_key
0,10006,142345,0,NaN,NaN,39.18,5032094
1,10006,142345,0,NaN,NaN,172.89,5032094
2,10006,142345,0,NaN,NaN,NaN,5032094
3,10011,105331,1,NaN,NaN,332.42,5062090
4,10011,105331,1,NaN,NaN,NaN,5062090



🧑‍⚕️ PATIENT


,subject_id,dob,gender,insurance,language,religion,marital_status,ethnicity
0,10006,2094-03-05 00:00:00,F,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE
1,10011,2090-06-05 00:00:00,F,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE
2,10013,2038-09-03 00:00:00,F,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE
3,10017,2075-09-21 00:00:00,F,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE
4,10019,2114-06-20 00:00:00,M,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE



💸 TYPE DE COUT


,drug,drug_type,dose_val_rx,dose_unit_rx,amount,cost_id
0,Pneumococcal Vac Polyvalent,MAIN,0.5,mL,400.0,1
1,Pneumococcal Vac Polyvalent,MAIN,0.5,mL,NaN,3
2,Pneumococcal Vac Polyvalent,MAIN,0.5,mL,200.0,5
3,Pneumococcal Vac Polyvalent,MAIN,0.5,mL,30.0,7
4,Pneumococcal Vac Polyvalent,MAIN,0.5,mL,NaN,9


In [2]:
# 📦 Imports communs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer

# 📊 Chargement et fusion des données nécessaires (df_facture, df_admission, df_patient)
df = pd.merge(df_facture, df_admission, on=['hadm_id', 'subject_id'], how='left')
df = pd.merge(df, df_patient, on='subject_id', how='left')

# 🎯 Cible binaire : dépassement du coût seuil
df['exceeds_cost'] = (df['total_cost'] > 10000).astype(int)

# 🧓 Calcul de l'âge
df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
df['age'] = pd.to_datetime('today').year - df['dob'].dt.year

# 🧹 Colonnes manquantes à initialiser
for col in ['procedure_count', 'admission_count']:
    if col not in df.columns or df[col].isna().all():
        df[col] = 0
# 🔁 Encodage des variables catégorielles
label_encoders = {}
for col in ['gender', 'insurance', 'marital_status']:
    df[col] = df[col].astype(str).fillna('Unknown')
    le = LabelEncoder()
    df[col + '_encoded'] = le.fit_transform(df[col])
    label_encoders[col] = le

# 🎯 Sélection des features
features = ['facility_cost', 'procedure_cost', 'medication_cost', 'lab_test_cost',
            'length_of_stay', 'procedure_count', 'admission_count', 'age',
            'gender_encoded', 'insurance_encoded', 'marital_status_encoded']
X = df[features]
y = df['exceeds_cost']

# 🧽 Imputation + Standardisation
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 🧪 Split stratifié avec minimum 1 instance/classe dans test
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, test_idx in sss.split(X, y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# 📊 Dictionnaire pour stocker les scores
model_scores = {}
# --- 4. SVM ---
from sklearn.svm import SVC
svm = SVC(kernel='rbf', C=1, class_weight='balanced', probability=True, random_state=42)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
model_scores['SVM'] = accuracy_score(y_test, y_pred_svm)
print("\n--- SVM ---")
print("Accuracy:", model_scores['SVM'])
print(classification_report(y_test, y_pred_svm, zero_division=0))


--- SVM ---
Accuracy: 0.9361702127659575
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.94      0.97        47

    accuracy                           0.94        47
   macro avg       0.50      0.47      0.48        47
weighted avg       1.00      0.94      0.97        47



In [4]:
import pickle

# Sauvegarder le StandardScaler
with open('scaler_classification.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Sauvegarder le SimpleImputer
with open('imputer_classification.pkl', 'wb') as f:
    pickle.dump(imputer, f)

# Sauvegarder le modèle SVM
with open('svm_model.pkl', 'wb') as f:
    pickle.dump(svm, f)
